## Defining and creating tensors

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
m1 = [[1.0, 2.0, 3.0],[4.0, 5.0, 6.0]]
m2 = np.array([[1.0, 2.0],[3.0, 4.0]], dtype=np.float32)
m3 = tf.constant([[1.0, 2.0, 3.0],[4.0, 5.0, 6.0]])

In [3]:
print type(m1)
print type(m2)
print type(m3)

<type 'list'>
<type 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [4]:
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)

In [5]:
print type(t1)
print type(t2)
print type(t3)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


### All operations in Tensorflow work on these tensors, and all inbuilt methods usually do use tf.convert_to_tensor to convert the data type to tensors before manipulating them.

In [6]:
m1 = tf.constant([[1., 2.]], name='m1')
m2 = tf.constant([[1],[2]], name='m2')
m3 = tf.constant([ [1,2], [3,4], [5,6], [7,8], [9,10]])

In [7]:
print m1
print m2
print m3

Tensor("m1:0", shape=(1, 2), dtype=float32)
Tensor("m2:0", shape=(2, 1), dtype=int32)
Tensor("Const_3:0", shape=(5, 2), dtype=int32)


### Each tensor object has a unique label name, a dimension shape, and dtype. Since we did not explicitly specify a name for the last tensor, it got name like Const_x:0, etc.

### Let's create a tensor of 500 x 500, each element having a value of 0.5

In [8]:
all_half = tf.ones([500,500])/2.0
print all_half

Tensor("div:0", shape=(500, 500), dtype=float32)


## Manipulating tensors with operators

In [9]:
x = tf.constant([[1,2]], dtype=tf.float32)

In [10]:
x

<tf.Tensor 'Const_4:0' shape=(1, 2) dtype=float32>

In [11]:
neg_x = tf.negative(x)
print neg_x

Tensor("Neg:0", shape=(1, 2), dtype=float32)


### Creating a normal distribution

In [12]:
from math import pi
mu = 0.0
sigma = 1.0
# Gaussian distribution operator
normal_dist = tf.exp(tf.negative(tf.pow((x - mu), 2))/(2.0*tf.pow(sigma, 2)))*\
            (1.0/ sigma*tf.sqrt(2.0*pi))

In [13]:
with tf.Session() as sess:
    result = sess.run(tf.negative(x))
    print(result)

[[-1. -2.]]


### Starting an interactive session so that session variable is not needed everytime we want to evaluate something

In [14]:
sess = tf.InteractiveSession()
result = neg_x.eval()
print result
sess.close()

[[-1. -2.]]


### Passing logging parameter in Session

In [15]:
with tf.Session(config= tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(neg_x)
    print sess.graph.device
print (result)

<bound method Graph.device of <tensorflow.python.framework.ops.Graph object at 0x7f159785cf90>>
[[-1. -2.]]


## Using variables

### A simple way of detecting spikes using a "spike" boolean variable

In [16]:
sess = tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spike = tf.Variable(False)
spike.initializer.run()

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike, True)
        updater.eval()
    else:
        tf.assign(spike, False).eval()
    print ("Spike: ", spike.eval())
sess.close()

('Spike: ', False)
('Spike: ', True)
('Spike: ', False)
('Spike: ', False)
('Spike: ', True)
('Spike: ', False)
('Spike: ', True)


### Storing spike's result in an array of booleans

In [17]:
sess = tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spikes = tf.Variable([False] * len(raw_data), name='spikes')
spikes.initializer.run()
tf.initialize_all_variables().run()
print spikes
# saver = tf.train.Saver()
saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        spikes_val = spikes.eval()
        spikes_val[i] = True
        updater = tf.assign(spikes, spikes_val)
        updater.eval()
save_path = saver.save(sess, "spikes.ckpt")
print("spikes data saved in file: %s" % save_path)
sess.close()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Tensor("spikes/read:0", shape=(8,), dtype=bool)
spikes data saved in file: spikes.ckpt


### Loading the variable back using 'restore' (Why is this not working?)

In [18]:
# sess = tf.InteractiveSession()
# spikes = tf.Variable([False]*8, name='spikes')
# # spikes.initializer.run() // No need to intialize this as it will be directly loaded from a file
# % ls
# saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
# saver.restore(sess,  "./spikes.ckpt")
# # print (spikes.eval())
# sess.close()

## Visualizing using TensorBoard

### Exponetial average computation

In [19]:
raw_data = np.random.normal(10, 1, 100)
alpha = tf.constant(0.05)
curr_value = tf.placeholder(tf.float32)
prev_avg = tf.Variable(0.)
update_avg = curr_value*alpha + (1-alpha)*prev_avg

# Defining summary variables to visualize on tensorboard
avg_hist = tf.summary.scalar("running_average", update_avg)
value_hist = tf.summary.scalar("incoming_values", curr_value)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("./logs")
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(len(raw_data)):
        summary_str, curr_avg = sess.run([merged, update_avg], feed_dict={curr_value: raw_data[i]})
        sess.run(tf.assign(prev_avg, curr_avg))
        print (raw_data[i], curr_avg)
        writer.add_summary(summary_str, i)

(8.8800489750340521, 0.44400245)
(9.7694742352608799, 0.91027606)
(11.40535558763948, 1.43503)
(8.3150537587015734, 1.7790313)
(10.325360959060404, 2.2063477)
(8.9661082070697304, 2.5443356)
(10.510751253048543, 2.9426565)
(10.396789158540059, 3.3153632)
(7.5562475292470843, 3.5274074)
(11.026650185486346, 3.9023695)
(9.9460384858335082, 4.2045531)
(10.204471753083192, 4.504549)
(9.8253066421865789, 4.770587)
(10.727404460444887, 5.068428)
(10.683146347238683, 5.349164)
(10.145887432381031, 5.5889997)
(9.9559561271761527, 5.8073478)
(9.0681759310577394, 5.9703889)
(8.9920922789490643, 6.1214738)
(9.405991007956402, 6.2856998)
(10.449226938890156, 6.493876)
(10.671681547347607, 6.7027664)
(8.7153621778433941, 6.8033962)
(9.3220451377305, 6.9293284)
(11.829598464381851, 7.1743417)
(10.933151906122243, 7.3622823)
(11.433676799385111, 7.5658522)
(10.122640867835576, 7.6936917)
(9.7154825671915894, 7.7947812)
(10.447503989899868, 7.9274173)
(9.6732473835399215, 8.0147085)
(10.08988898178467

In [20]:
raw_data

array([  8.88004898,   9.76947424,  11.40535559,   8.31505376,
        10.32536096,   8.96610821,  10.51075125,  10.39678916,
         7.55624753,  11.02665019,   9.94603849,  10.20447175,
         9.82530664,  10.72740446,  10.68314635,  10.14588743,
         9.95595613,   9.06817593,   8.99209228,   9.40599101,
        10.44922694,  10.67168155,   8.71536218,   9.32204514,
        11.82959846,  10.93315191,  11.4336768 ,  10.12264087,
         9.71548257,  10.44750399,   9.67324738,  10.08988898,
        10.13119703,  10.89401442,  10.81547564,   7.58287384,
         9.90455801,   9.47671984,   9.61254789,  10.19943057,
         9.97381801,   9.31344421,   9.08839558,  10.04055752,
         8.76894012,  10.55966024,  10.37721096,  10.86081556,
         9.74006833,   8.29311389,   9.7739748 ,  11.68036448,
         9.17670639,   9.52065446,   8.16113843,   9.99400165,
        11.71838087,   9.07040183,  10.28512551,  11.18631765,
        10.44532163,   8.90738285,  11.72584732,   8.50